In [6]:
# 손실함수
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0) # 오버플로를 막기 위해서 exp안에 최댓값을 빼줌 
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로를 막기 위해서 exp안에 최댓값을 빼줌 
    return np.exp(x) / np.sum(np.exp(x))


# 배치용 교차 엔트로피 오차 
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    delta = 1e-7  # 1e-7은 log안에 0이 되지 않도록 더해주는 값
    
    # 배치의 크기로 나눠 정규화 하고 평균 교차 엔트로피 오차를 계산 
    return -np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size

def sigmoid(x) :
    return 1 / (1+np.exp(-x))

In [7]:
# 활성화 함수 

class Relu:
    def __init__(self):
        self.mask = None # mask : T/F로 구성된 배열

    def forward(self, x):
        self.mask = (x <= 0) # 0보다 같거나 작으면 True(1)
        out = x.copy()
        out[self.mask] = 0 # fancy index : out[x<=0] = 0 -> 0보다 작은 원소들을 0으로 

        return out

    def backward(self, dout): 
        dout[self.mask] = 0   # 역전파시 0보다 작은 원소들 0으로    
        dx = dout

        return dx

# sigmoid : y = 1 / (1+exp(-x))
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out   # 역전파 시 y(1-y) 곱해줌

        return dx


# Wx + b
# 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서 affine transformation이라고 함 
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

In [8]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.layers import *
# from common.gradient import numerical_gradient
from collections import OrderedDict



class MLP:

    def __init__(self, input_size, hidden_size_1,hidden_size_2, output_size, weight_init_std = 0.01):
        
        # 층 3개  
        self.params = {} # 딕셔너리 변수
        
        # 1번째 층의 가중치와 편향
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size_1)
        self.params['b1'] = np.zeros(hidden_size_1)
        # 2번쨰 층의 가중치와 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size_1, hidden_size_2) 
        self.params['b2'] = np.zeros(hidden_size_2)
        # 3번째 층의 가중치와 편향
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size_2, output_size)
        self.params['b3'] = np.zeros(output_size)
        
      
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
      #  self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
      #  self.layers['Sigmoid2'] = Sigmoid()
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    # 예측 함수 
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
        
    # x : 입력 데이터, t : 정답 레이블
    # 손실함수의 값을 구함 
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t) # SoftmaxWithLoss y,t의 cross_entropy
    
    # 정확도 
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
        
    # x : 입력 데이터, t : 정답 레이블
#     def numerical_gradient(self, x, t):
#         loss_W = lambda W: self.loss(x, t)
        
#         grads = {}
#         grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
#         grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
#         grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
#         grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
#         return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1                             # 1로 초기화 
        dout = self.lastLayer.backward(dout) # lastLayer : softmax 부터 
        
        layers = list(self.layers.values()) # layer들의 리스트 
        layers.reverse()
        for layer in layers:                  # 역전파 
            dout = layer.backward(dout) 

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W3'], grads['b3'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        
        return grads

In [11]:
# 입력층 784, 출력층 10 
from data.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(60000, 784) (60000, 10)
(10000, 784) (10000, 10)


In [12]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from data.mnist import load_mnist

# 데이터 mnist데이터 셋에서 onehot데이터 load
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = MLP(input_size=784, hidden_size_1=256, hidden_size_2=128, output_size=10)


epochs = 100

idx_arr = np.arange(len(x_train))

batch_size = 1000
learning_rate = 0.1

for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list),
                                                                                         train_acc*100, test_acc*100))

Epoch : 0 / Loss : 2.3013 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 1 / Loss : 2.2983 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 2 / Loss : 2.2875 / Train Acc : 21.6200% / Test Acc : 21.7800%
Epoch : 3 / Loss : 2.1598 / Train Acc : 33.6833% / Test Acc : 33.6100%
Epoch : 4 / Loss : 1.5045 / Train Acc : 66.5883% / Test Acc : 67.5500%
Epoch : 5 / Loss : 0.8305 / Train Acc : 77.2683% / Test Acc : 78.0100%
Epoch : 6 / Loss : 0.6564 / Train Acc : 79.8433% / Test Acc : 80.6100%
Epoch : 7 / Loss : 0.5686 / Train Acc : 84.4167% / Test Acc : 84.6300%
Epoch : 8 / Loss : 0.4988 / Train Acc : 85.9283% / Test Acc : 86.2400%
Epoch : 9 / Loss : 0.4544 / Train Acc : 87.3700% / Test Acc : 87.4800%
Epoch : 10 / Loss : 0.4217 / Train Acc : 87.9650% / Test Acc : 88.0300%
Epoch : 11 / Loss : 0.3955 / Train Acc : 88.8233% / Test Acc : 88.7700%
Epoch : 12 / Loss : 0.3727 / Train Acc : 89.5550% / Test Acc : 89.3700%
Epoch : 13 / Loss : 0.3526 / Train Acc : 89.9883% / Test Acc : 89.7500%
Ep

In [64]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from data.mnist import load_mnist

# 데이터 mnist데이터 셋에서 onehot데이터 load
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = MLP(input_size=784, hidden_size_1=50, hidden_size_2 = 25,output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)   # train_size = 60000, batch_size = 100, 60000개 중 임의 100개 
    x_batch = x_train[batch_mask]   
    t_batch = t_train[batch_mask] 
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch) 
    train_loss_list.append(loss) 
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train accuracy:", train_acc,"test accuracy:", test_acc)

train accuracy: 0.10218333333333333 test accuracy: 0.101
train accuracy: 0.4860333333333333 test accuracy: 0.4879
train accuracy: 0.8671 test accuracy: 0.8727
train accuracy: 0.9166333333333333 test accuracy: 0.9151
train accuracy: 0.93705 test accuracy: 0.934
train accuracy: 0.9472833333333334 test accuracy: 0.9456
train accuracy: 0.95495 test accuracy: 0.9519
train accuracy: 0.96425 test accuracy: 0.9581
train accuracy: 0.9664166666666667 test accuracy: 0.9609
train accuracy: 0.9686166666666667 test accuracy: 0.9616
train accuracy: 0.9732166666666666 test accuracy: 0.9659
train accuracy: 0.97435 test accuracy: 0.966
train accuracy: 0.97755 test accuracy: 0.9688
train accuracy: 0.9775166666666667 test accuracy: 0.9675
train accuracy: 0.9807 test accuracy: 0.968
train accuracy: 0.9837166666666667 test accuracy: 0.9716
train accuracy: 0.9835 test accuracy: 0.9681


In [20]:
train_size = x_train.shape[0]
batch_size = 100
np.random.choice(train_size, batch_size)
train_size

60000

In [63]:
np.random.choice(5,1)

array([1])